In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np

tf.config.list_physical_devices()

2023-11-12 20:52:51.125414: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 20:52:51.156979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 20:52:51.157017: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 20:52:51.157039: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 20:52:51.163149: I tensorflow/core/platform/cpu_feature_g

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
print ("importing successful")
train = pd.read_csv('train.csv')

print ("train dataset column")
print(train)

importing successful
train dataset column
       BookingID  LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  \
0              1        10         2018             3           31   
1              2       116         2018             2           28   
2              3        11         2018             7           25   
3              4         3         2017             9           12   
4              5        28         2018             3            7   
...          ...       ...          ...           ...          ...   
29015      29016         2         2018            11           21   
29016      29017        18         2018             4           12   
29017      29018         8         2017             9           29   
29018      29019        20         2018             9           18   
29019      29020       274         2018             7           31   

       NumWeekendNights  NumWeekNights      MealPlan  Parking     RoomType  \
0                     0              1 

In [3]:
train.dropna(inplace=True)

# Encode categorical features
label_encoder = LabelEncoder()
train['BookingStatus'] = label_encoder.fit_transform(train['BookingStatus'])
train['MarketSegment'] = label_encoder.fit_transform(train['MarketSegment'])
train['MealPlan'] = label_encoder.fit_transform(train['MealPlan'])
train['RoomType'] = label_encoder.fit_transform(train['RoomType'])

# Add new feature 'leadTime'
train['leadTime'] = train['LeadTime'] / 365.0

In [4]:
"""# Feature Engineering """

# Separate features and target variable
# Separate features and target variable
X = train[['ArrivalMonth', 'AvgRoomPrice', 'MarketSegment', 'MealPlan', 'NumAdults', 'NumChildren', 'NumPrevCancellations', 'NumPreviousNonCancelled', 'NumWeekNights', 'NumWeekendNights', 'RepeatedGuest', 'RoomType', 'SpecialRequests', 'leadTime']]
y = train['BookingStatus']

# Split data into train and test sets
# Split the data into training, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.75, random_state=42)


# Normalize features
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [5]:
"""# Model Selection"""

# Create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],), kernel_regularizer = tf.keras.regularizers.l1(0.01)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

# Compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3840      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 32)                2

2023-11-12 20:53:07.801715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-12 20:53:07.801984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-12 20:53:07.802102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
# Train model
# time taken to train the model

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, shuffle=True, batch_size=100, verbose=1)


Epoch 1/40


2023-11-12 20:53:12.984987: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f98b4db78c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-12 20:53:12.985029: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2023-11-12 20:53:12.988719: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-12 20:53:12.999853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-11-12 20:53:13.069365: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


59/59 [==============================] - 3s 10ms/step - loss: 2.7525 - accuracy: 0.7669 - val_loss: 3.7967 - val_accuracy: 0.3456
Epoch 2/40
59/59 [==============================] - 0s 7ms/step - loss: 1.7733 - accuracy: 0.8244 - val_loss: 3.1367 - val_accuracy: 0.3469
Epoch 3/40
59/59 [==============================] - 0s 7ms/step - loss: 1.2338 - accuracy: 0.8341 - val_loss: 4.9234 - val_accuracy: 0.3455
Epoch 4/40
59/59 [==============================] - 0s 7ms/step - loss: 1.0234 - accuracy: 0.8258 - val_loss: 7.2810 - val_accuracy: 0.3453
Epoch 5/40
59/59 [==============================] - 0s 7ms/step - loss: 0.8825 - accuracy: 0.8355 - val_loss: 6.3792 - val_accuracy: 0.3469
Epoch 6/40
59/59 [==============================] - 0s 7ms/step - loss: 0.7480 - accuracy: 0.8434 - val_loss: 9.7983 - val_accuracy: 0.3479
Epoch 7/40
59/59 [==============================] - 0s 7ms/step - loss: 0.6686 - accuracy: 0.8455 - val_loss: 6.5192 - val_accuracy: 0.3793
Epoch 8/40
59/59 [============

In [7]:
"""# Model Evaluation

## Mode Accuracy, f1-score and recall
"""

# #Evaluate model
y_pred = model.predict(X_test)
# #print (y_pred)
y_pred = (y_pred > 0.5).astype(int)

# #Define target names
target_names = ['Class 1', 'Class 2']

# # Generate classification report
print(classification_report(y_test, y_pred, target_names=target_names))

# Evaluate the model on the test set
accuracy = accuracy_score(y_pred, y_test) * 100

print("Test Accuracy 2:", accuracy)

182/182 [==============================] - 0s 911us/step
              precision    recall  f1-score   support

     Class 1       0.76      0.72      0.74      1892
     Class 2       0.87      0.89      0.88      3912

    accuracy                           0.83      5804
   macro avg       0.81      0.80      0.81      5804
weighted avg       0.83      0.83      0.83      5804

Test Accuracy 2: 83.3907649896623


In [9]:
"""## Predictions for Test dataset

### Import/Format the Test dataset
"""
test_dataset_location = 'test.csv'
test = pd.read_csv(test_dataset_location)
export_test = pd.read_csv(test_dataset_location)

output_location = 'test_output.csv'
output = pd.read_csv(output_location)

# Encode categorical features
label_encoder = LabelEncoder()
test['BookingStatus'] = label_encoder.fit_transform(test['BookingStatus'])
test['MarketSegment'] = label_encoder.fit_transform(test['MarketSegment'])
test['MealPlan'] = label_encoder.fit_transform(test['MealPlan'])
test['RoomType'] = label_encoder.fit_transform(test['RoomType'])

# Add new feature 'leadTime'
test['leadTime'] = test['LeadTime'] / 365.0

# Normalize features
mean = test.mean(axis=0)
std = test.std(axis=0)
test = (test - mean) / std
test = (test - mean) / std

print (test)

test_x = test[['ArrivalMonth', 'AvgRoomPrice', 'MarketSegment', 'MealPlan', 'NumAdults', 'NumChildren', 'NumPrevCancellations', 'NumPreviousNonCancelled', 'NumWeekNights', 'NumWeekendNights', 'RepeatedGuest', 'RoomType', 'SpecialRequests', 'leadTime']]

# Formating output file
output['BookingStatus'] = label_encoder.fit_transform(output['BookingStatus'])

output_y = output['BookingStatus']
print(output_y)

FileNotFoundError: [Errno 2] No such file or directory: 'test_output.csv'

In [ ]:
# #Evaluate model
y_pred = model.predict(test_x)
y_pred = (y_pred > 0.5).astype(int)

# #print (y_pred)

# #Define target names
target_names = ['Class 1', 'Class 2']

# # Generate classification report
print(classification_report(output_y, y_pred, target_names=target_names))